In [1]:
# Imports
import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.tokenize import sent_tokenize as st
import pandas as pd
import spacy


In [2]:
nlp = spacy.load('en_core_web_md')

In [3]:
#nlp = spacy.load('en_core_web_md')

In [4]:
#spacy__version__

In [5]:
#sample_sentences = "C:/Users/dinesh.panchal/Desktop/CA Folder/Rated_sentences.xlsx" # Enter directory in place of '.'
characteristics = "BM_Chars.csv"
company = "AR_sample.csv"
export_path = "Results4.xlsx"

In [6]:
## Defining stop words
stop_words = ['a', '.', ',', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
              'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him',
              'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its',
              'itself', 'they', 'them', 'their', 'theirs', 'themselves',
              'what', 'which', 'who', 'whom', 'this', 'that', 'these',
              'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
              'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
              'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because',
              'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
              'against', 'between', 'into', 'through', 'during', 'before', 'after',
              'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
              'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
              'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',
              'few', 'more', 'most', 'other', 'some', 'such',
              'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
              'will', 'just', 'don','should', 'now','https',"'s",'...', "whats'",
              "rt","whats","n't","de","'m","un","en","``","dedic","twittermoments",
              "amp","e","y","o","ce","retweet","sur","na","el","1","2","3","4",
              "5","6","7","8","9","0","ca","nao","se","com","los","u","des","-",
              "--","'","''","c","a",
              "b","d","e","f","g","h","i","j","k","l","m","n","o",
              "p","q","r","s","t","u","v","w","x","y","z"]

In [7]:
# Evaluates Similarity between sentences using WordNet path similarity
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize
    sent1 = ' '.join([x.lower() for x in wt(sentence1) if x.lower() not in stop_words])
    sent2 = ' '.join([x.lower() for x in wt(sentence2) if x.lower() not in stop_words])
    
    synsets1 = nlp(sent1)
    synsets2 = nlp(sent2)

    score, count = 0.0, 0

    for synset in synsets1:
        try:
            # print(synset)
            best_score = []
            for ss in synsets2:
                best_score.append(synset.similarity(ss))
            # print(best_score)
            best_score.append(0)
            best_score = max(x for x in best_score if x is not None)
            # best_score = max([synset.path_similarity(ss) for ss in synsets2])
        except:
            print("Error: Couldn't do scoring")
            best_score = None
        # Check that the similarity could have been computed
        if best_score is not None:
            score = score + best_score
        count = count + 1
    if count != 0:
        score = score / count
        return score
    else:
        # print("Error: Can't divide by count = 0")
        return 0

In [8]:
# Two way similarity measure: Main function, call this to evaluate similarity
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1))/2

In [1]:
# Testing technique with LnT data

## Import Business Model Characteristics
chars = pd.read_csv(characteristics, encoding='latin1')

## Import LnT Annual Report paragraphs
lnt = pd.read_excel(company,  encoding='latin1')


# Begin Timer
import time
start_time = time.time()


## Parse paragraphs into individual sentences
lnt_sent = []

for i in range(len(lnt)):
    lnt_sent.append(st(str(lnt.iloc[i, 0])))

full_lnt = [sent for sublist in lnt_sent for sent in sublist]

df_lnt = pd.DataFrame({'Text': full_lnt})


# Evaluate similarity of sentences with individual characteristics

for i in range(len(chars)):
    characteristics = []
    n=100

    for row in full_lnt:
        s1_d1=row[0:n]
        s1_d2=row[n:2*n]
        s1_d3=row[2*n:3*n]
        
        s2_d1=chars.iloc[i,0][0:n]
        s2_d2=chars.iloc[i,0][n:2*n]
        s2_d3=chars.iloc[i,0][2*n:3*n]
        characteristics.append(euclidean_similarity(str(s1_d1), str(s2_d1)))
        characteristics.append(euclidean_similarity(str(s1_d2), str(s2_d2)))
        characteristics.append(euclidean_similarity(str(s1_d3), str(s2_d3)))
    df_lnt[i] = characteristics
    print(i)

#Export results into an Excel file
df_lnt.to_excel(export_path)

# Time Elapsed
print("\nTime elapsed: {:.2f}s".format(time.time() - start_time))